In [1]:
import meteomatics.api as mapi
import datetime as dt
import sqlalchemy
import pandas as pd

In [2]:
#username = raw_input("Enter username:")


In [3]:
#password = raw_input("Enter pass:")
username = 'python-community'
password = 'Umivipawe179'

In [4]:
startdate_ts = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0) 
enddate_ts = startdate_ts + dt.timedelta(days=7)
interval_ts = dt.timedelta(hours=1)
#coordinates_ts = [(47.25,9.34), (30,-4)]
london=[(51.5072,0.1276)] #London
athens=[(37.9838,23.7275)]#Athens
singapore=[(1.3521,103.8198)]
parameters_ts = ['t_2m:C','precip_1h:mm']

In [29]:
def api_call(country,countryname):
    try:
        df_ts = mapi.query_time_series(country, startdate_ts, enddate_ts, interval_ts,
                                      parameters_ts, username, password)
        df_clean=df_ts.reset_index()
        df_clean['Country']=countryname
        df_clean['day_name']=df_clean.validdate.dt.day_name()
        df_clean['day']=df_clean.validdate.dt.day_of_week
        df_clean['hour']=df_clean.validdate.dt.hour
        df_clean= df_clean.rename({'t_2m:C': 'temp', 'precip_1h:mm': 'precip'}, axis='columns')
        return df_clean
    except Exception as e:
        print("The API Call Failed, the exception is {}".format(e))

In [30]:
lon_fc=api_call(london,"London")
ath_fc=api_call(athens,"Athens")
sng_fc=api_call(singapore,"Singapore")


In [32]:
dbengine=sqlalchemy.create_engine('sqlite:///C:\\sqlite\\margera_assignment.db')

In [33]:
lon_fc.to_sql('forecast', dbengine, if_exists='fail')

In [34]:
ath_fc.to_sql('forecast', dbengine, if_exists='append')

In [35]:
sng_fc.to_sql('forecast', dbengine, if_exists='append')

In [9]:
dbengine.execute('DROP TABLE forecast').commit()

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: DROP TABLE forecast]
(Background on this error at: https://sqlalche.me/e/14/e3q8)